# Building a Database for Crime Reports

PostgreSQL is useful for working with large datasets.

In this project I will demonstrate how to build a PostgreSQL database from [this Boston crime data here](https://www.kaggle.com/ankkur13/boston-crime-data).

In [21]:
import psycopg2
import csv
import cchardet

### Establishing the Connection

In [9]:
conn = psycopg2.connect(dbname='###', user='###')
conn.autocommit = True
cur = conn.cursor()

### Creating the Database

In [ ]:
cur.execute("CREATE DATABASE crime_db;")

conn.close() # Closing the connection

### Connecting to the `crime_db` Database

In [10]:
conn = psycopg2.connect(dbname='crime_db', user='###')
conn.autocommit = True
cur = conn.cursor()

cur.execute("CREATE SCHEMA crimes;") # Creating the necessary schema

### Opening the `crime.csv` file as a list of lists

In [18]:
with open('crime.csv', mode='rb') as file: # Checking encoding
    raw_bytes = file.read()
    
encoding = cchardet.detect(raw_bytes)['encoding']

with open('crime.csv', encoding=encoding) as file: # Opening with encoding
    
    reader = csv.reader(file)
    boston = list(reader)
    
    for i in range(len(boston)): # Selecting columns for database
        row = boston[i]
        boston[i] = [row[0], row[1], row[3], row[7], row[10], row[14], row[15]]
    
    col_headers = boston[0] # Header row
    first_row = boston[1] # Data rows

In [41]:
# Viewing header row
col_headers

['INCIDENT_NUMBER',
 'OFFENSE_CODE',
 'OFFENSE_DESCRIPTION',
 'OCCURRED_ON_DATE',
 'DAY_OF_WEEK',
 'Lat',
 'Long']

In [42]:
# Viewing first data row
first_row

['I182080058',
 '02403',
 'DISTURBING THE PEACE',
 '2018-10-03 20:13:00',
 'Wednesday',
 '42.26260773',
 '-71.12118637']

### Writing the `boston` list of lists to a `.csv` file

In [54]:
with open('boston.csv', mode='w', newline='') as file:
    
    writer = csv.writer(file)
    writer.writerows(boston)

### Identifying proper datatypes for the columns

When creating our database, we want to make sure the values in each column are entered as the most optimal datatypes.

We want to find out:

* If any columns should be enumerated datatypes
* How many characters each varchar column should allow

To come to the first conclusion, we can build a function that determines how many unique values are in each column. If there are very few, that column can be enumerated.

To calculate the answers for the second task, we can use the same function that we build to determine the longest character length in any potential varchar columns.

In [45]:
# Function that returns a set of unique values for any column

def get_col_value_set(csv_filename, col_index):
    """
    Creates a set of unique values from a given column in a dataset
    
    Args:
        csv_filename (list): list of lists, dataset to search
        col_index (int): column from which to generate set
        
    Returns:
        set
    """
    unique_values = set()
    for i in range(len(csv_filename)): 
        unique_values.add(csv_filename[i][col_index])
        
    return unique_values

In [46]:
# Calculating number of unique values in each column
for j in range(len(col_headers)):
    
    print(len(get_col_value_set(boston[1:], j)))

290156
263
244
239364
7
18240
18240


It looks like column 4, `day_of_week`, should be enumerated.

Additionally, we should have 2 varchar columns, from column 0 and column 2.

Let's see what the max length is in these columns:

In [83]:
# Determining max character length for column 0, `incident_number`
max_length=0

for value in get_col_value_set(boston[1:], 0):
    max_length = max(max_length, len(value))
    
max_length

13

In [84]:
# Determining max character length for column 2, `description`
max_length=0

for value in get_col_value_set(boston[1:], 2):
    max_length = max(max_length, len(value))
    
max_length

58

### Creating enumerated values for `day_of_week` column

In [50]:
cur.execute("""
    CREATE TYPE day_of_week_enum AS ENUM 
    
    ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday');
    
    """)

### Creating the `boston_crimes` data table with proper datatypes

In [99]:
cur.execute("""
    CREATE TABLE crimes.boston_crimes (
    
        incident_number VARCHAR(30),
        offense_code SMALLINT,
        description VARCHAR(100),
        date DATE,
        day_of_the_week DAY_OF_WEEK_ENUM,
        latitude DECIMAL(10,8),
        longitude DECIMAL(10,8)
        
        );
    """)

### Loading the `boston.csv` data into the `crimes.boston_crimes` datatable

In [100]:
with open('boston.csv') as file:
    
    cur.copy_expert("""
    COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;
    """, file)

### Revoking `public` group privileges

In [101]:
cur.execute("""
    REVOKE ALL ON SCHEMA public FROM public;
    """)

cur.execute("""
    REVOKE ALL ON DATABASE crime_db FROM public;
    """)

### Creating `readonly` and `readwrite` groups

In [102]:
cur.execute("""
    CREATE GROUP readonly NOLOGIN;
    CREATE GROUP readwrite NOLOGIN;
    """)

In [103]:
cur.execute("""
    GRANT CONNECT ON DATABASE crime_db TO readonly;
    GRANT CONNECT ON DATABASE crime_db TO readwrite;
    """)

In [104]:
cur.execute("""
    GRANT USAGE ON SCHEMA crimes TO readonly;
    GRANT USAGE ON SCHEMA crimes TO readwrite;
    """)

### Granting privileges

The `readonly` group can only select the data. The `readwrite` group can make changes to the data. I will give the groups privileges that reflect this:

In [105]:
cur.execute("""
    GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;
    """)

In [106]:
cur.execute("""
    GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;
    """)

### Creating Users

In [107]:
cur.execute("""
    CREATE USER data_analyst WITH PASSWORD 'password1';
    
    GRANT readonly TO data_analyst;
    """)

In [108]:
cur.execute("""
    CREATE USER data_scientist WITH PASSWORD 'password2';
    
    GRANT readwrite TO data_scientist;
    """)

### Closing the connection

In [110]:
conn.close()

### Testing the Configuration

I will now test to ensure the database is configured correctly:

In [111]:
# Testing connection to new database
conn = psycopg2.connect(dbname="crime_db", user="admin")
cur = conn.cursor()

In [113]:
# Testing for users and groups
cur.execute("""
    SELECT  rolname, 
            rolsuper, 
            rolcreaterole, 
            rolcreatedb, 
            rolcanlogin 
            
        FROM pg_roles
    
        WHERE rolname IN ('readonly', 'readwrite', 'data_analyst', 'data_scientist');
""")

for user in cur:
    print(user)
print()

('readonly', False, False, False, False)
('readwrite', False, False, False, False)
('data_analyst', False, False, False, True)
('data_scientist', False, False, False, True)



In [114]:
# Testing for privileges
cur.execute("""
    SELECT  grantee, 
            privilege_type
            
        FROM information_schema.table_privileges
    
        WHERE grantee IN ('readonly', 'readwrite');
""")

for user in cur:
    print(user)

('readonly', 'SELECT')
('readwrite', 'INSERT')
('readwrite', 'SELECT')
('readwrite', 'UPDATE')
('readwrite', 'DELETE')


In [115]:
conn.close()

## Conclusion

We now have a database of Boston Crime data that we can view and/or update depending on user group privileges. 

### Next Steps:

* Create indexes for querying the data